In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd ###
from shapely.geometry import Point, Polygon###
from fiona.drvsupport import supported_drivers###
import folium as fm
import re
%matplotlib inline

#Crime dataframe
crime_df = pd.read_csv('2020_FBI_Crime_Statistics.csv')
crime_df = crime_df.fillna(0) #fill in Nan with 0's
crime_df['Crime Index'].astype('float64')


#Zip Code data frame
supported_drivers['KML'] = 'rw'
place_df = gpd.read_file('./Geographies/DC_MD_VA_Places.kml', driver='KML')

def find_city_name(cell):
    match = re.search(r'<at><openparen>([A-Za-z ]+)<closeparen>', cell)
    if match:
        return match.group(1)
    else:
        return None

cities = list()
for r in place_df.iterrows():
    cities.append(find_city_name(r[1]['Name']))
    
place_df['City'] = cities

#Merging crime and zip DataFrames. Not that not every city in zip has reported crime data to FBI data base
crime_df_merge = pd.merge(place_df, crime_df[['City','Crime Index']], on='City', how = 'left')

crime_df_merge = crime_df_merge.fillna(0) #filling Nan with 0's

#setting cities without reported crime to the average of the population reported by FBI
crime_df_merge.loc[crime_df_merge['Crime Index']==0, ['Crime Index']] = 4.46 
display(crime_df_merge.head())


,Name,Description,geometry,City,Crime Index
0,<at><openparen>Washington<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.11976 38.93434 0.00000, -77.11...",Washington,10.45
1,<at><openparen>Aberdeen<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.20372 39.53569 0.00000, -76.20...",Aberdeen,5.52
2,<at><openparen>Aberdeen Proving Ground<closepa...,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.17136 39.49442 0.00000, -76.17...",Aberdeen Proving Ground,4.46
3,<at><openparen>Abingdon<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.29964 39.46149 0.00000, -76.29...",Abingdon,3.15
4,<at><openparen>Accident<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-79.32468 39.62597 0.00000, -79.32...",Accident,4.46


In [23]:
crime_df.head()

,State,City,Population,Violent,Murder,Rape,Robbery,Assault,Property,Burglary,Theft,Vehicle,Arson,Total Crimes,Crime Index
0,Virginia,Abingdon,7933.0,7,0,1,1,5,118,6,104,8,0.0,250,3.15
1,Virginia,Alexandria,162258.0,288,2,19,80,187,2517,117,2005,395,2.0,5612,3.46
2,Virginia,Amherst,2179.0,3,0,1,0,2,14,3,11,0,0.0,34,1.56
3,Virginia,Appalachia,1544.0,6,0,2,0,4,48,8,34,6,1.0,109,7.06
4,Virginia,Ashland,7916.0,19,0,1,4,14,158,8,142,8,1.0,355,4.48


In [27]:
pentagon = (38.8719570149196, -77.05630981703996)
pentagon_grid = Point(pentagon[1],pentagon[0])
#Once you have your metric loaded on your GeoDataFrame above, you can use the code below to start 
#to visualize. We will work more on the fillcolor and popups later.
zip_map = fm.Map(location=pentagon, zoom_start=12, tiles='CartoDB positron')
fm.Marker(pentagon, popup = 'Pentagon').add_to(zip_map)
for _, r in crime_df_merge.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fm.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    fm.Popup(str(r['Crime Index'])).add_to(geo_j)
    geo_j.add_to(zip_map)
zip_map